# 🛰️ Welcome to PoliSim

**An Open-Source Federal Fiscal Policy Simulator**

---

## What You'll Learn

In this notebook (15-20 minutes), you will:

1. **Understand** what PoliSim does and why it matters
2. **Run** your first fiscal policy simulation
3. **Interpret** basic budget projections
4. **Visualize** spending and revenue trends

## What is PoliSim?

PoliSim is an **open-source, transparent alternative** to the Congressional Budget Office's (CBO) fiscal projections. 

### The Problem
Federal fiscal decisions rely on closed-source models. Citizens, researchers, and policymakers can't independently verify assumptions or explore policy alternatives.

### The Solution
PoliSim provides:
- 🔍 **Transparency**: Every assumption documented and configurable
- 📊 **Monte Carlo Simulations**: Full uncertainty quantification (100K+ iterations)
- ✅ **Validated**: Projections within ±2-5% of CBO/SSA/CMS baselines
- 🌐 **Open Source**: MIT license - anyone can use, verify, and extend

---

Let's get started!

## 1. Setup: Import PoliSim

First, let's import the core modules and verify everything is working.

In [ ]:
# Add parent directory to path (for notebook execution)
import sys
from pathlib import Path

# Navigate to project root
notebook_dir = Path.cwd()
if notebook_dir.name == 'notebooks':
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir
    
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import PoliSim core modules
from core import (
    get_policy_by_type,
    PolicyType,
    simulate_healthcare_years,
    list_available_policies,
)

# Import visualization libraries
import pandas as pd
import matplotlib.pyplot as plt

# Verify import success
print("✅ PoliSim modules loaded successfully!")
print(f"📂 Project root: {project_root}")
print(f"\n📋 Available policy types: {[p.value for p in PolicyType]}")

## 2. Load a Policy

PoliSim includes **8 pre-built healthcare policy models**. Let's load the **USGHA** (US Global Health Assurance) policy - a comprehensive healthcare reform proposal.

| Policy | Description |
|--------|-------------|
| **USGHA** | US Global Health Assurance - comprehensive reform |
| **CURRENT_US** | Current US healthcare system baseline |
| **M4A** | Medicare-for-All single-payer |
| **UK_NHS** | UK National Health Service model |
| **CANADA** | Canadian single-payer system |
| **AUSTRALIA** | Australian hybrid model |
| **GERMANY** | German multi-payer system |
| **UN_FRAMEWORK** | UN healthcare framework |

In [ ]:
# Load the USGHA policy
policy = get_policy_by_type(PolicyType.USGHA)

# Display policy details
print(f"📋 Policy: {policy.policy_name}")
print(f"📝 Description: {policy.description[:150]}...")
print(f"\n🎯 Key Parameters:")
print(f"   • Coverage target: {policy.coverage_percentage:.0%}")
print(f"   • Healthcare spending target: {policy.healthcare_spending_target_gdp:.1%} of GDP")
print(f"   • Administrative overhead: {policy.admin_overhead_percentage:.1%}")

## 3. Run Your First Simulation

Now let's run a **10-year fiscal simulation** to see how this policy affects the federal budget.

**Key parameters:**
- **Base GDP**: $29 trillion (2025 estimate)
- **Initial Debt**: $35 trillion
- **Population**: 335 million
- **GDP Growth**: 2.5% annual

In [ ]:
# Run 10-year simulation
results = simulate_healthcare_years(
    policy=policy,
    base_gdp=29e12,           # $29 trillion GDP
    initial_debt=35e12,        # $35 trillion debt
    years=10,                  # 10-year projection
    population=335e6,          # 335 million people
    gdp_growth=0.025,          # 2.5% annual GDP growth
    start_year=2025
)

print(f"✅ Simulation complete!")
print(f"📊 Generated {len(results)} years of projections")
print(f"\n📈 Results preview:")

## 4. Understanding the Output

Let's examine what the simulation tells us. Key metrics include:

- **GDP**: Gross Domestic Product (total economic output)
- **Health Spending**: Total healthcare expenditure
- **Health % GDP**: Healthcare as a percentage of the economy
- **Revenue**: Total revenue collected
- **Surplus/Deficit**: Whether we're spending more or less than revenue

In [ ]:
# Select key columns for display
display_cols = ['Year', 'GDP', 'Health Spending ($)', 'Health % GDP', 'Revenue ($)', 'Surplus ($)']

# Filter to available columns
available_cols = [col for col in display_cols if col in results.columns]

# Format for readability
summary = results[available_cols].copy()

# Convert large numbers to trillions for readability
for col in summary.columns:
    if col not in ['Year', 'Health % GDP']:
        if summary[col].dtype in ['float64', 'int64']:
            summary[col] = summary[col] / 1e12  # Convert to trillions

# Rename columns
summary.columns = [col.replace(' ($)', ' ($T)') for col in summary.columns]

print("📊 10-Year Budget Projection ($ in Trillions)")
print("=" * 70)
print(summary.to_string(index=False, float_format=lambda x: f"{x:.2f}" if abs(x) > 0.01 else f"{x:.4f}"))

## 5. Visualizing Results

Charts help us understand trends over time. Let's create visualizations of key metrics.

In [ ]:
# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('USGHA Policy: 10-Year Fiscal Projection', fontsize=14, fontweight='bold')

# Plot 1: GDP Growth
ax1 = axes[0, 0]
ax1.plot(results['Year'], results['GDP'] / 1e12, 'b-', linewidth=2, marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('GDP ($ Trillions)')
ax1.set_title('GDP Growth')
ax1.grid(True, alpha=0.3)

# Plot 2: Healthcare Spending
ax2 = axes[0, 1]
ax2.plot(results['Year'], results['Health Spending ($)'] / 1e12, 'r-', linewidth=2, marker='s')
ax2.set_xlabel('Year')
ax2.set_ylabel('Spending ($ Trillions)')
ax2.set_title('Healthcare Spending')
ax2.grid(True, alpha=0.3)

# Plot 3: Healthcare as % of GDP
ax3 = axes[1, 0]
ax3.plot(results['Year'], results['Health % GDP'] * 100, 'g-', linewidth=2, marker='^')
ax3.set_xlabel('Year')
ax3.set_ylabel('% of GDP')
ax3.set_title('Healthcare as % of GDP')
ax3.axhline(y=18.5, color='gray', linestyle='--', label='2025 Baseline (18.5%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Revenue vs Spending
ax4 = axes[1, 1]
ax4.plot(results['Year'], results['Revenue ($)'] / 1e12, 'b-', linewidth=2, marker='o', label='Revenue')
ax4.plot(results['Year'], results['Health Spending ($)'] / 1e12, 'r-', linewidth=2, marker='s', label='Health Spending')
ax4.set_xlabel('Year')
ax4.set_ylabel('$ Trillions')
ax4.set_title('Revenue vs Healthcare Spending')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Notice how healthcare spending as % of GDP changes over time under this policy.")

## 6. Try It Yourself!

Now it's your turn. Try modifying the simulation parameters below and see how the results change.

In [ ]:
# 🎮 EXPERIMENT: Try changing these values!
# ============================================

# Choose a different policy (options: USGHA, CURRENT_US, M4A, UK_NHS, CANADA, AUSTRALIA, GERMANY)
my_policy = get_policy_by_type(PolicyType.CURRENT_US)  # Try changing this!

# Adjust economic assumptions
my_gdp_growth = 0.025  # Try: 0.01 (low), 0.025 (baseline), 0.04 (high)
my_years = 15          # Try: 10, 15, 22

# Run your simulation
my_results = simulate_healthcare_years(
    policy=my_policy,
    base_gdp=29e12,
    initial_debt=35e12,
    years=my_years,
    population=335e6,
    gdp_growth=my_gdp_growth,
    start_year=2025
)

# Compare first and last year
first_year = my_results.iloc[0]
last_year = my_results.iloc[-1]

print(f"📋 Policy: {my_policy.policy_name}")
print(f"📈 GDP Growth: {my_gdp_growth:.1%}")
print(f"📅 Projection Years: {my_years}")
print(f"\n🔄 Change from Year 1 to Year {my_years}:")
print(f"   • GDP: ${first_year['GDP']/1e12:.1f}T → ${last_year['GDP']/1e12:.1f}T")
print(f"   • Health Spending: ${first_year['Health Spending ($)']/1e12:.2f}T → ${last_year['Health Spending ($)']/1e12:.2f}T")
print(f"   • Health % GDP: {first_year['Health % GDP']:.1%} → {last_year['Health % GDP']:.1%}")

## 🎓 What You've Learned

Congratulations! You've completed your first PoliSim session. You now know how to:

✅ Import and use PoliSim modules  
✅ Load healthcare policy models  
✅ Run multi-year fiscal simulations  
✅ Interpret budget projection results  
✅ Visualize fiscal trends  

---

## 📚 Next Steps

Continue your learning journey with these notebooks:

| Notebook | Topic | Time |
|----------|-------|------|
| **02** | Federal Budget Basics | 30-40 min |
| **03** | Healthcare Policy Analysis | 45-60 min |
| **04** | Social Security Deep Dive | 45-60 min |
| **05** | Monte Carlo & Uncertainty | 60-90 min |
| **06** | Tax Policy Modeling | 45-60 min |

---

## 🔗 Resources

- **Documentation**: [documentation/INDEX.md](../documentation/INDEX.md)
- **Dashboard**: Run `python launcher.py` to start the interactive dashboard
- **API**: See [docs/API_ENDPOINTS.md](../docs/API_ENDPOINTS.md) for programmatic access
- **GitHub**: [github.com/GalacticOrgOfDev/polisim](https://github.com/GalacticOrgOfDev/polisim)

---

*Thank you for exploring PoliSim! Together, we're democratizing fiscal policy analysis.* 🛰️